In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
def load_data(name):
    if name == 'tdep':
        return pd.read_pickle('saved/dep_data.pkl')
    if name == 'kahovka':
        return pd.read_pickle('saved/kahovka_deposits.pkl')
    if name == 'wind':
        return pd.read_pickle('saved/wind_data.pkl')
    if name == 'rain':
        return pd.read_pickle('saved/rain_data.pkl')
    raise ValueError("Data not defined")

In [3]:
wind_data = load_data('wind')
rain_data = load_data('rain')
kahovka = load_data('kahovka')

In [4]:
first_wind = []
for cell_data in wind_data.wind_history[0]:
    first_wind.append(cell_data[0])
first_wind = np.array(first_wind)
first_wind.shape


(8056, 2)

---

## NN

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
TOTAL_CELLS_NUM = 8056
SMALLER_CELLS_NUM = 1900

device = 'cpu'

In [ ]:
# TODO: implement dataset

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [13]:
# Define model (https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(SMALLER_CELLS_NUM, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1900, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")